# BAMBOO: Binary descriptor based on AsymMetric pairwise BOOsting

In this notebook we include the implementation of the BAMBOO descriptor to provide a compressed representation of probe requests.

## Libraries and Configurations

Import configuration files

In [13]:
from configparser import ConfigParser

config = ConfigParser()
config.read("../config.ini")

['../config.ini']

Import **data libraries**

In [14]:
import pandas as pd

Import **other libraries**

In [15]:
from rich.progress import Progress
from rich import traceback

traceback.install()

from tqdm.notebook import tqdm

In [16]:
import numpy as np

## Import Data

In [17]:
# Combined dataframe
combined_df_csv = (
    config["DEFAULT"]["interim_path"] + "dissected/std_burst_dissected_df.csv"
)

In [18]:
df = pd.read_csv(combined_df_csv, index_col=0)

In [19]:
df

,MAC Address,Channel,DS Channel,Vendor Specific Tags,Length,Label,Supported Rates 1,Supported Rates 2,Supported Rates 3,Supported Rates 4,...,TIM_Broadcast,BSS_Transition,Multiple_BSSID,Timing_Measurement,SSID_List,DMS,Interworking,QoS_Map,WNM_Notification,Operating_Mode_Notification
0,00:0f:00:6a:68:8b,1,NaN,2,279,SamsungJ6_K,65.0,66.0,69.5,75.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02:00:00:00:00:00,11,9.0,11,123,SamsungM31_A,1.0,2.0,5.5,11.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,02:00:00:00:3e:b2,11,11.0,62,132,iPhone11_C,65.0,66.0,69.5,75.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,02:00:3a:5e:a1:f4,11,10.0,62,132,iPhone11_B,65.0,66.0,69.5,75.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5,02:02:70:30:b6:43,1,3.0,62,143,iPhone12_W,65.0,66.0,69.5,75.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4410,fe:f9:ac:47:0d:b7,11,11.0,62,131,iPhone12_W,65.0,66.0,69.5,75.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4411,fe:f9:fc:fb:83:9e,6,1.0,1,156,iPhone6_N,1.0,2.0,5.5,11.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4412,fe:fc:07:34:10:69,1,1.0,62,132,iPhone11_C,65.0,66.0,69.5,75.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4413,fe:fc:aa:d1:89:d1,1,2.0,62,143,iPhone12_W,65.0,66.0,69.5,75.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


## BAMBOO

Input:
- Ground truth relationships $\langle x_{a(n)}, x_{b(n)}; y_n\rangle$
  - $n=1,..,N$
  - $y_n \in \{+1, -1\}$
- A set of filters $\mathcal{H} = \{h_1 , ..., h_F\}$
- A set of binarization thresholds $\mathcal{T} = \{t_1 , ..., t_T\}$

Output:
- A set of $M<F$ filters $[h_{i(1)}, ..., h_{i(M)}]$
- Corresponding set of binarization thresholds $[t_{j(1)}, ..., t_{j(M)}]$

### Filters

In [20]:
import itertools

In [21]:
def generate_filters(num_columns: int) -> list:
    # Generate all combinations of 1s and 0s for the number of columns
    combinations = list(itertools.product([0, 1], repeat=num_columns))
    # Remove the combination that includes only 0s (all columns excluded)
    combinations.remove((0,) * num_columns)

    filter_vectors = []
    for combination in tqdm(combinations):
        filter_vector = [1 if include else 0 for include in combination]
        filter_vectors.append(filter_vector)

    return filter_vectors

In [22]:
# # Esempio di utilizzo
# num_columns = 4
# column_names = ['col1', 'col2', 'col3', 'col4']
# data = {
#     'col1': [1, 2, 3],
#     'col2': [4, 5, 6],
#     'col3': [7, 8, 9],
#     'col4': [10, 11, 12]
# }
# df = pd.DataFrame(data)

# # Print the filter vectors
# for i, filter_vector in enumerate(generate_filters(num_columns)):
#     print(f"Filter {i+1}: {filter_vector}")

In [23]:
len(df.columns)

46

In [25]:
filters = generate_filters(23)

  0%|          | 0/8388607 [00:00<?, ?it/s]